<a href="https://colab.research.google.com/github/RichardLin3/InfoSysProj/blob/main/ISProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

repo_url = "https://github.com/RichardLin3/InfoSysProj.git"
repo_dir = "InfoSysProj"

if os.path.exists(repo_dir) and os.listdir(repo_dir):
    print(f"Directory '{repo_dir}' already exists and is not empty. Skipping git clone.")
else:
    print(f"Cloning '{repo_url}' into '{repo_dir}'...")
    !git clone {repo_url}

Cloning 'https://github.com/RichardLin3/InfoSysProj.git' into 'InfoSysProj'...
Cloning into 'InfoSysProj'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 63 (delta 28), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (63/63), 86.03 KiB | 3.91 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [2]:
import json

with open('InfoSysProj/data/gened_v2.json', 'r') as f:
    gened_data = json.load(f)

with open('InfoSysProj/data/all-majors.json', 'r') as f:
    major_data = json.load(f)

with open('InfoSysProj/data/minors_v3.json', 'r') as f:
    minor_data = json.load(f)

with open('InfoSysProj/data/liberal_arts_courses.json', 'r') as f:
    lib_data = json.load(f)

with open('InfoSysProj/data/ID.json', 'r') as f:
    ID_data = json.load(f)

In [3]:
major_names = list(major_data.keys())
minor_names = list(minor_data.keys())

In [4]:
#major selection
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=major_names,
    description='Select a Major:',
    disabled=False,
)

def on_change(change):
    global selected_major
    if change['type'] == 'change' and change['name'] == 'value':
        selected_major = change['new']
        print(f"You selected: {selected_major}")

dropdown.observe(on_change)

display(dropdown)

Dropdown(description='Select a Major:', options=('Data Science', 'Applied Chemistry BS', 'Architecture Technol…

In [5]:
dropdown2 = widgets.Dropdown(
    options=minor_names,
    description='Select a Minor:',
    disabled=False,
)

def on_change2(change):
    global selected_minor
    if change['type'] == 'change' and change['name'] == 'value':
        selected_minor = change['new']
        print(f"You selected: {selected_minor}")

dropdown2.observe(on_change2)

display(dropdown2)

Dropdown(description='Select a Minor:', options=('Arabic Language and Cultural Studies', 'Art History & Visual…

In [6]:
if 'selected_major' not in globals() or selected_major is None:
  selected_major = major_names[0]
if 'selected_minor' not in globals() or selected_minor is None:
  selected_minor = minor_names[0]

In [7]:
courses = []  # Initialize an empty list to store course codes
minor_electives = []

minor_info = minor_data[selected_minor]
for section_name, section_list in minor_info.items(): # Iterate through the items of the minor_info dictionary
    # section list is usually a list
    for entry in section_list:
        # Check if the entry is a dictionary and has a 'Course' key
        if isinstance(entry, dict) and 'Course' in entry:
            courses.append(entry['Course'])
            if section_name == 'Electives':
              minor_electives.append(entry['Course'])
        # Check if the entry is a dictionary and contains nested course options (like 'Choose 1')
        elif isinstance(entry, dict):
            for key, value in entry.items():
                if isinstance(value, list):
                    for item in value:
                        if isinstance(item, dict) and 'Course' in item:
                            courses.append(item['Course'])
                            if section_name == 'Electives':
                                minor_electives.append(item['Course'])


print(courses) # Print the extracted courses to verify

['ARB 2201', 'ARB 1101', 'ARB 1102', 'Any other ARB course', 'ARCH 1121', 'ARTH 1101', 'ARTH 1110', 'HIS 1501', 'HIS 1502', 'HIS 1503', 'MUS 1212', 'SOC 1104']


In [8]:
gened_areas = ['EC', 'MQR', 'LPS', 'WCGI', 'USED', 'IS', 'CE', 'SW' ,'Lib Art', 'Adv Lib Art', 'ID', 'Major_Core','Major_Elec']
course_gened_fulfillment_list = []

In [9]:
print(minor_names)

['Arabic Language and Cultural Studies', 'Art History & Visual Culture', 'Black Visual Cultures', 'Business', 'Computer Science', 'Creative Writing', 'Environmental Studies', 'Gender & Sexuality Studies', 'Hispanic Studies', 'History', 'Physics', 'Psychology', 'Spanish Language', 'Theatre Studies']


In [11]:
#for future use of the number of minor electives need to be taken
#minor_elect_num = [{'Arabic Language and Cultural Studies':2}, {'Art History & Visual Culture':3}, {'Black Visual Cultures':3}, {'Business':1},
#                   {'Computer Science':2}, {'Creative Writing':1}, {'Environmental Studies':4}, {'Gender & Sexuality Studies':3},
#                   {'Hispanic Studies':2}, {'History':1}, {'Physics':2}, {'Psychology':4}, {'Spanish Language':2}, {'Theatre Studies':1}
#]

In [12]:
gened_fulfillment_map = {}
for core_type in ['Required Core', 'Flexible Core']:
    for course_info in gened_data['gened'][core_type]:
        areas = course_info.get('Area(s)', [])
        gened_fulfillment_map[course_info['Course']] = areas

In [13]:
#set of course in the core and electives for followup processing
major_core = set([course['Course'] for course in major_data[selected_major]['Core']])
major_elective = set([course['Course'] for course in major_data[selected_major]['Electives']])
Adv_lib = set([course['Course'] for course in lib_data['Adv Lib Art']])
Lib_art = set([course['Course'] for course in lib_data['Lib Art']])

In [14]:
import re

course_gened_fulfillment_list = [] # Clear the list before populating it

for course in courses:
    fulfilled_geneds = gened_fulfillment_map.get(course, [])

    if not isinstance(fulfilled_geneds, list):
            fulfilled_geneds = [fulfilled_geneds] if fulfilled_geneds else []

    # Check for major core and elective fulfillment
    if course in major_core:
        fulfilled_geneds.append('Major_Core')
    if course in major_elective:
        fulfilled_geneds.append('Major_Elec')

    # Check for Liberal Arts fulfillment
    if course in Lib_art:
        fulfilled_geneds.append('Lib Art')
    if course in Adv_lib:
        fulfilled_geneds.append('Lib Art')
        fulfilled_geneds.append('Adv Lib Art')

    # Check for ID fulfillment
    if course in ID_data or re.search(r'\d+ID$', course):
        fulfilled_geneds.append('ID')


    course_gened_fulfillment_list.append({
        'Courses in Minor': course,
        'Fulfilled Areas': fulfilled_geneds
    })

In [15]:
import pandas as pd

course_fulfillment_list_for_df = []
gened_areas_for_columns = gened_areas

for course_info in course_gened_fulfillment_list:
    course_name = course_info['Courses in Minor']
    fulfilled_areas = course_info['Fulfilled Areas']
    course_dict = {'Courses in Minor': course_name}
    for gened_abbr in gened_areas_for_columns:
        course_dict[gened_abbr] = 'X' if gened_abbr in fulfilled_areas else ''
    course_fulfillment_list_for_df.append(course_dict)

course_fulfillment_df = pd.DataFrame(course_fulfillment_list_for_df)
course_fulfillment_df = course_fulfillment_df.fillna('')

# Set the index to start from 1
course_fulfillment_df.index = range(1, len(course_fulfillment_df) + 1)

# Display the DataFrame with dotted horizontal and vertical lines and dynamic caption
# display(
#     course_fulfillment_df.style
#     .set_properties(**{'border': '1px dotted black'})
#     .set_caption(f'{selected_major} Major and {selected_minor} Minor Course Fulfillment Overview')
#     .set_table_styles([{'selector': 'caption', 'props': [('font-size', '20px')]}]
#     )
# )

In [16]:
full_df = course_fulfillment_df.copy()

# 1. Separate the 'Courses in Minor' column and drop it from the data columns
# The column name is 'Courses in Minor' as per the latest notebook state.
course_names_series = full_df['Courses in Minor']
full_df = full_df.drop(columns=['Courses in Minor'])

# --- Dynamic Grouping Logic from minor_data for the selected minor ---
# This dictionary will store course_code -> group_rule
course_group_lookup = {}

minor_sections = minor_data[selected_minor]
for section_name, items_list in minor_sections.items():
    for item in items_list:
        # Case 1: Course is part of an explicit "group" key (e.g., "group": "Choose 1")
        if "group" in item:
            group_rule = item["group"]
            for course_dict in item["courses"]:
                if "Course" in course_dict:
                    course_group_lookup[course_dict["Course"]] = group_rule

# 2. Prepare the Index DataFrame for the MultiIndex
old_idx_data = {'Course Name': course_names_series.tolist()}

# 3. Define the new top index level ('Index Group') based on the dynamic lookup
index_group_level = []
for course_code in course_names_series:
    # Get the group from the lookup, default to 'Uncategorized Requirement' if not found
    group = course_group_lookup.get(course_code, "")
    index_group_level.append(group)

# 4. Insert the new top index level into a temporary DataFrame to create MultiIndex
old_idx_df = pd.DataFrame(old_idx_data)
old_idx_df.insert(0, 'Groups', index_group_level)

# 5. Create the final MultiIndex and assign it back to the DataFrame
full_df.index = pd.MultiIndex.from_frame(old_idx_df)

# Display the Final Result (The display logic is in the next cell 'fv8tT04FnNSn')

In [17]:
def color_minor_courses(row):
    # Initialize an empty list of styles, one for each cell in the row
    styles = [''] * len(row)
    course_name = row.name[1]  # Get the 'Course Name' level of the MultiIndex

    # First, apply row-level styling for core/elective distinction
    row_background_color = ''
    if course_name in minor_electives:
        row_background_color = 'lightblue'
    else:
        row_background_color = 'lightgreen'

    # Apply the row-level background color to all cells first
    if row_background_color:
        styles = [f'background-color: {row_background_color}'] * len(row)

    # Now, add cell-specific styling (e.g., highlighting 'X' marks)
    for i, (col_name, cell_value) in enumerate(row.items()):
        if col_name in gened_areas and cell_value == 'X':
            styles[i] += '; color: darkred; font-weight: bold'

    return styles

def color_index_courses(idx_labels_series):
    # This function will receive a Series of index labels for the level specified in apply_index
    styles = []
    for label in idx_labels_series:
        if label in minor_electives:
            styles.append('background-color: lightblue')
        else:
            styles.append('background-color: lightgreen')
    return styles

styled_df = full_df.style \
    .apply(color_minor_courses, axis=1) \
    .apply_index(color_index_courses, axis=0, level='Course Name') \
    .set_properties(**{'border': '1px dotted black'}) \
    .set_caption(f'{selected_major} Major and {selected_minor} Minor Course Fulfillment Overview') \
    .set_table_styles([
        {'selector': 'caption', 'props': [('font-size', '20px')]},
        {'selector': 'th.row_heading', 'props': [('border', '1px dotted black')]}
    ])
display(styled_df)